# TCP

<img src='res/tcp01.png' width=500px />
<img src='res/tcp02.png' width=500px />

## 缩写

congestion control algorithm(CCA), congestion window(CWND), receiver's advertised window(RWND), maximum segment size(MSS), slow start threshold(ssthresh), round-trip delay/time(RTD,RTT), Retransmission Timeout(RTO)

--------





## slow start,congestion control,fast retransmit,fast recovery

<img src='res/tcp03.png' width=500px />

[RFC2001](res/rfc2001.pdf)

<img src='res/tcp04.png' width=500px />

--------

## linux2.6/include/net/tcp.h

```c
/*
 * The next routines deal with comparing 32 bit unsigned ints
 * and worry about wraparound (automatic with unsigned arithmetic).
 */

static inline int before(__u32 seq1, __u32 seq2)
{
        return (__s32)(seq1-seq2) < 0;
}

static inline int after(__u32 seq1, __u32 seq2)
{
	return (__s32)(seq2-seq1) < 0;
}


/* is s2<=s1<=s3 ? */
static inline int between(__u32 seq1, __u32 seq2, __u32 seq3)
{
	return seq3 - seq2 >= seq1 - seq2;
}
```

1. 比较tricky的比较两个无符号sequence number的办法。主要的问题是seq num可能会wrap around，比如4bits的值，seq1是1111,seq2是0，seq1 应该小于 seq2，如果直接比较是不对的

2. before函数为例， seq1-seq2 = seq1 + seq2的补码 = seq1 + （(1<<32)-seq2）= seq1-seq2 + (1<<32)，如果没有发生wraparound，值是对的。如果发生wraparound，seq1-seq2就是两个数之间的距离，只要(距离>=1^31)，那么符号位就是1，就是个负数，符号也是对的。所以这个函数有效，必须wrapround发生的距离不太大，即seq1,seq2的差值>=1^31。实际seq num是线性递增的，1^31是个很大的数，一般不会超过

3. between函数，与before类似，两个数相减计算的永远是这两个数之间的距离(正数就是距离，负数就是补码距离=1^32+负数)，无论是否wraparound。所以只要3到2的距离大于1到2的距离，那么1就在2和3之间

Very nice and tricky method !!!

-------------------------